<a href="https://colab.research.google.com/github/mralamdari/CV-Yolo/blob/main/YOLO_v3_ObjectDetection_TensorFlow_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import cv2
import PIL
import struct # used to convert native Python data types such as strings and numbers into a string of bytes and vice versa
import scipy.io
import scipy.misc
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.layers.merge import add, concatenate

In [ ]:
os.environ['KAGGLE_CONFIG_DIR'] = '/content/drive/MyDrive'
!kaggle datasets download -d aruchomu/data-for-yolo-v3-kernel
!unzip \*.zip && rm *.zip

 96% 257M/267M [00:09<00:00, 24.8MB/s]
100% 267M/267M [00:09<00:00, 29.0MB/s]
Archive:  data-for-yolo-v3-kernel.zip
  inflating: coco.names              
  inflating: detections.gif          
  inflating: dog.jpg                 
  inflating: futur.ttf               
  inflating: office.jpg              
  inflating: yolov3.weights          


In [5]:


class Weights_Reader:

  def __init__(self, filename):

    with open(filename, 'rb') as w_file:
      major, = struct.unpack('i', w_file.read(4))
      minor, = struct.unpack('i', w_file.read(4))
      revision, = struct.unpack('i', w_file.read(4))

      if (major*10 + minor) >= 2 and major < 1000 and minor < 1000:
        w_file.read(8)
      else:
        w_file.read(4)
      
    self.offset = 0
    self.total_weights = np.frombuffer(w_file.read(), dtype='float32')

  def byte_reader(self, size):
    self.offset = self.offset + size
    return self.total_weights[self.offset - size: self.offset]
  
  def load_weights(self, model):

    for i in range(106): # 53*2 = 106 layers in total
      try: # if it is a Convolutional Layer
        convolutional_layer = model.get_layer(f'conv_{i}')
        print(f'Loading weights of Convolution Layer {i}')

        if i not in [81, 93, 105]: #if it is not a Detection Layer (82, 94, 106)
          batchnorm_layer = model.get_layer(f'bnorm_{i}')
          size = np.prod(batchnorm_layer.get_weights()[0].shape) 

          bias = self.byte_reader(size)
          scale = self.byte_reader(size)
          mean = self.byte_reader(size)
          variance = self.byte_reader(size)

          weights = batchnorm_layer.set_weights([bias, scale, mean, variance])

        kernel = self.byte_reader(np.prod(convolutional_layer.get_weights()[0].shape))
        kernel = kernel.reshape(list(reversed(convolutional_layer.get_weights()[0].shape)))
        kernel = kernel.transpose([2, 3, 1, 0])

        if len(convolutional_layer.get_weights()) > 1:
          bias = self.byte_reader(np.prod(convolutional_layer.get_weights()[1].shape))
          convolutional_layer.set_weights([kernel, bias])
        else:
          convolutional_layer.set_weights([kernel])

      except ValueError:
        print(f"Layer {i} is Not Convolution Layer")

        
  def reset(self):
    self.offset = 0        